In [1]:
from numba import cuda, vectorize, float32
from scipy import sparse
import random
import math
import string

import time
import numpy as np

In [2]:
TPB = 4

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [3]:
#The A and B matrices must be multiples of TPB?

In [4]:
num_of_chars = TPB*4

A = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(TPB*1)])
B = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(TPB*2)])

In [5]:
# The data array
#A = np.full((TPB*3, TPB*5), 3, np.float) # [32 x 48] matrix containing all 3's
#B = np.full((TPB*5, TPB*2), 4, np.float) # [48 x 16] matrix containing all 4's

In [6]:
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B.T)
C_global_mem = cuda.device_array((A.shape[0], B.shape[0])) 

In [7]:
# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[0] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [8]:
blockspergrid

(1, 2)

In [9]:
# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res = C_global_mem.copy_to_host()

In [10]:
res

array([[2., 3., 2., 2., 2., 4., 5., 4.],
       [3., 5., 3., 5., 5., 6., 5., 6.],
       [1., 1., 3., 2., 3., 2., 2., 3.],
       [1., 3., 3., 3., 2., 4., 4., 3.]])

In [11]:
np.matmul(A, B.T)

array([[2, 3, 2, 2, 2, 4, 5, 4],
       [3, 5, 3, 5, 5, 6, 5, 6],
       [1, 1, 3, 2, 3, 2, 2, 3],
       [1, 3, 3, 3, 2, 4, 4, 3]])